# City name search

Customer: Yandex Practicum Career Center

## Project desription 
<a name="project-descr"></a>

**Goal**

- Mapping arbitrary geo names to uniform geonames for internal use by the Career Center

**Objectives**

- Create a solution to match the most appropriate names with geonames. For example Ереван -> Yerevan

- On the example of RF and countries most popular for relocation - Belarus, Armenia, Kazakhstan, Kyrgyzstan, Turkey, Serbia. Cities with population from 15000 people (with the possibility of scaling on the customer's server).

- Returned fields geonameid, name, region, country, cosine similarity

- output data format: list of dictionaries, e.g. [{dict_1}, {dict_2}, .... {dict_n}] where dictionary is one record with specified fields

*Optional*:

- possibility to customize the number of matching names output (e.g. in method parameters)

- correction of errors and misprints. For example Моченгорск -> Monchegorsk

- storage of geonames data in PostgreSQL

- storing vectorized intermediate data in PostgreSQL

- provide methods for configuring connection to the database

- provide methods for class initialization (primary vectorization of geonames)

- provide methods for adding vectors of new geonames

**Implementation period**

Tentative timeframe for the project is 3 weeks from 11/28/2023 

## Data sources

* [geonames.org](http://download.geonames.org/export/dump/)
* [Test dataset](https://disk.yandex.ru/d/wC296Rj3Yso2AQ) 

----------------------------



# Approach

There are multple ways to compare text similarity. Toponyms (like city names) hold only scarce semantic relations with each other and with the common words ('dog', 'summer', 'theater'), so advanced language models (like sentence transformers) do not expected to provide major benifits, if they are feasible at all. On the other hand, word similarity estimates algorithms like Levenstein distance and cosine distanse are relatively simple yet powerful techiques that are robust to misspellings. `Fuzzy` search is the fast implementation of computation of Levenstein distance and, unlike cosine similarity, does not require additional encoding steps. 

**Research plan**
- Preparation: set up the PostgreSQL engine and re-create the customer's database 
- Implementation of the solution based on `fuzzy` search
- ??? Trying out other matching algorithms based on charachter embeddings 

The matches are ranged by
1. The similarity score with the main or any of the alteratve names -  **MVP**
   - Where the current official name is given priority
2. In the rare event of equality of the scores range the cities by
   - population 
   - admnistrative significance

As such, the necessary columns are:

**`cities15000`** or
**`geonames`** (composed from source tables **`XX.txt`**)
- `geonameid`
- `name`
- `asciiname`
- `alternatenames`
- `population`
- `admin1_code`
- `country_code`

For the region:
**`admin1CodesASCII`**
- `code`
- `name`

**`countryInfo`**
- `ISO`
- `Country`
- `Capital` (?)

For the extra features:

**`alternateNamesV2`**
- `alternate name`
- `isPreferredName`
- `isShortName`
- `isColloquial`
- `isHistoric` - to have an option to exclude / include the historical names. 
- `admin2_code`
- `admin3_code`
- `admin4_code` (?)



# 1. Preparations
## 1.1 Connecting to the database

In [1]:
## Installing depedencies

# %pip install pandas numpy sqlalchemy>=2.0.23 psycopg2 python-dotenv transliterate

In [1]:
import pandas as pd
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine, MetaData, Table, Column, select, Integer, String, DECIMAL, CHAR, BIGINT, func, DATE
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.sql import text

from dotenv import load_dotenv # The sensitive info about the database connection is stored in the .env file
import os
load_dotenv()  

True

In [2]:
import sqlalchemy
sqlalchemy.__version__ # Important to have SQLAlchemy > 2.0!

'2.0.23'

In [2]:
# Read the following from the environment variables
USR = os.getenv('USR') # Tip: NEVER name an env variable "USERNAME"
PWD = os.getenv('PWD')
DB_HOST = os.getenv('DB_HOST')
PORT = os.getenv('PORT')
DB = os.getenv('DB')

DATABASE = {
    'drivername': 'postgresql',
    'username': USR,
    'password': PWD,
    'host': DB_HOST,
    'port': PORT,
    'database': DB, 
    'query': {}
}

# Creating an Engine object
engine = create_engine(URL.create(**DATABASE))

# Checking the connection
try:
    # Подключаемся к базе данных
    with engine.connect() as conn:
         # Trying to execute a simple test query. The `text` function converst a string into and SQL-query
        result = conn.execute(text("SELECT 1"))
        for _ in result:
            pass  # don't do anything
    print(f"Connection established: {DATABASE['database']} на {DATABASE['host']}")
except SQLAlchemyError as e:
    print(f"Connection error: {e}")

Connection established: geo_v2 на 77.222.36.33


## 1.2 Initializing the data on the dev side
This part is left here for demonstration, reproduibility and consistency. The customer already have their DB set up.

Dataset specifications are taken from [geonames.org](https:\\geonames.org) and followed as is when creating the database. 
From all the data presented there, for an 

Many of the columns are easy to read for the users, they are not convenient to handle in queries and scripts. Despite that, they are *not* renamed for compatibility with the customer's database. 

### `countryInfo`

In [92]:
column_names= ['ISO', 'ISO3', 'ISO-Numeric', 'fips', 'Country', 'Capital',
       'Area(in sq km)', 'Population', 'Continent', 'tld', 'CurrencyCode',
       'CurrencyName', 'Phone', 'Postal Code Format', 'Postal Code Regex',
       'Languages', 'geonameid', 'neighbours', 'EquivalentFipsCode']

data = pd.read_csv('../datasets/countryInfo.txt', skiprows=50, sep='\t', index_col=None, names=column_names, encoding='utf-8')
#data=data.rename(columns={"#ISO": "ISO"})
data.head()

,ISO,ISO3,ISO-Numeric,fips,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",NaN
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NaN,.ag,XCD,Dollar,+1-268,NaN,NaN,en-AG,3576396,NaN,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN,.ai,XCD,Dollar,+1-264,NaN,NaN,en-AI,3573511,NaN,NaN


In [ ]:
# Uploading the table to the database
data.to_sql('countryInfo', con=engine, if_exists='replace', index=False) # mind the "replace" option!

In [119]:
metadata = MetaData()
cinfo = Table('countryInfo', metadata,
    Column('ISO', CHAR(2)),
    Column('ISO3', CHAR(3)),
    Column('ISO-Numeric', Integer),
    Column('fips', CHAR(2)),
    Column('Country', String(200)),
    Column('Capital', String(200)),    
    Column('Area(in sq km)', DECIMAL),
    Column('Population', BIGINT),
    Column('Continent', CHAR(2)),
    Column('tld', CHAR(3)),
    Column('CurrencyName', CHAR(3)),
    Column('Phone', String(30)),
    Column('Postal Code Format', String(30)),
    Column('Postal Code Regex', String(100)),
    Column('Languages', String(100)),
    Column('geonameid', Integer),
    Column('neighbours', String(200)),
    Column('EquivalentFipsCode', CHAR(2))
)

metadata.create_all(engine)

In [98]:
column_names= ['ISO', 'ISO3', 'ISO-Numeric', 'fips', 'Country', 'Capital',
       'Area(in sq km)', 'Population', 'Continent', 'tld', 'CurrencyCode',
       'CurrencyName', 'Phone', 'Postal Code Format', 'Postal Code Regex',
       'Languages', 'geonameid', 'neighbours', 'EquivalentFipsCode']

data = pd.read_csv('../datasets/countryInfo.txt', skiprows=49, sep='\t', index_col=None, encoding='utf-8', usecols=['#ISO', 'ISO3', 'Country', 'Capital', 'Population', 'Continent', 'Languages', 'geonameid'])
#data=data.rename(columns={"#ISO": "ISO"})
data.head()

,#ISO,ISO3,Country,Capital,Population,Continent,Languages,geonameid
0,AD,AND,Andorra,Andorra la Vella,77006,EU,ca,3041565
1,AE,ARE,United Arab Emirates,Abu Dhabi,9630959,AS,"ar-AE,fa,en,hi,ur",290557
2,AF,AFG,Afghanistan,Kabul,37172386,AS,"fa-AF,ps,uz-AF,tk",1149361
3,AG,ATG,Antigua and Barbuda,St. John's,96286,NaN,en-AG,3576396
4,AI,AIA,Anguilla,The Valley,13254,NaN,en-AI,3573511


In [99]:
# Data upload
data.to_sql('countryInfo2', con=engine, if_exists='append', index=False)

252

In [7]:
query = 'SELECT * FROM "countryInfo2" LIMIT 10'
pd.read_sql_query(query, con=engine)

,#ISO,ISO3,Country,Capital,Population,Continent,Languages,geonameid
0,AD,AND,Andorra,Andorra la Vella,77006,EU,ca,3041565
1,AE,ARE,United Arab Emirates,Abu Dhabi,9630959,AS,"ar-AE,fa,en,hi,ur",290557
2,AF,AFG,Afghanistan,Kabul,37172386,AS,"fa-AF,ps,uz-AF,tk",1149361
3,AG,ATG,Antigua and Barbuda,St. John's,96286,None,en-AG,3576396
4,AI,AIA,Anguilla,The Valley,13254,None,en-AI,3573511
5,AL,ALB,Albania,Tirana,2866376,EU,"sq,el",783754
6,AM,ARM,Armenia,Yerevan,2951776,AS,hy,174982
7,AO,AGO,Angola,Luanda,30809762,AF,pt-AO,3351879
8,AQ,ATA,Antarctica,None,0,AN,None,6697173
9,AR,ARG,Argentina,Buenos Aires,44494502,SA,"es-AR,en,it,de,fr,gn",3865483


### `alternateNamesV2` 

This table is a detailed version of the column `alternative` `geonames`
Note, that the dataset `alternateNames` is deprecated
One has to treat this dataset careful: the table is by far the heviest of all and the data is heterogeneous

In [4]:
data = pd.read_csv('../datasets/alternateNamesV2.txt', sep='\t', index_col=None, header=None)
data.head()

C:\Users\krair\AppData\Local\Temp\ipykernel_1600\1219458091.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../datasets/alternateNamesV2.txt', sep='\t', index_col=None, header=None)


,0,1,2,3,4,5,6,7,8,9
0,1284819,2994701,NaN,Roc Mélé,NaN,NaN,NaN,NaN,NaN,NaN
1,1284820,2994701,NaN,Roc Meler,NaN,NaN,NaN,NaN,NaN,NaN
2,4285256,3007683,NaN,Pic des Langounelles,NaN,NaN,NaN,NaN,NaN,NaN
3,1291197,3017832,NaN,Pic de les Abelletes,NaN,NaN,NaN,NaN,NaN,NaN
4,4290387,3017832,NaN,Pic de la Font-Nègre,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data.loc[data[1]==2017370] # Alternative names of Russian Federation 

,0,1,2,3,4,5,6,7,8,9
12105224,993186,2017370,en,Russian Soviet Federated Socialist Republic,NaN,NaN,NaN,1.0,NaN,NaN
12105225,993187,2017370,NaN,Rossiyskaya Sovetskaya Federativnaya Sotsialis...,NaN,NaN,NaN,1.0,NaN,NaN
12105226,993188,2017370,en,Russian Soviet Federative Socialist Republic,NaN,NaN,NaN,1.0,NaN,NaN
12105227,993191,2017370,en,Russian Socialist Federative Soviet Republic,NaN,NaN,NaN,1.0,NaN,NaN
12105228,1556474,2017370,aa,Russia,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12105386,16930770,2017370,wo,Risi,1.0,NaN,NaN,NaN,NaN,NaN
12105387,16930771,2017370,yi,רוסלאַנד,1.0,NaN,NaN,NaN,NaN,NaN
12105388,16930772,2017370,zh,俄罗斯,1.0,NaN,NaN,NaN,NaN,NaN
12105389,17433252,2017370,ru,России,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data.columns = ['alternateNameId', 'geonameid', 'isolanguage', 'alternate name', 'isPreferredName', 'isShortName', 'isColloquial', 'isHistoric', 'from', 'to']

In [5]:
engine.connect().rollback()

metadata = MetaData()
geonames = Table('alternateNames', metadata,
    Column('alternateNameId', Integer),
    Column('geonameid', Integer),
    Column('isolanguage', CHAR(7)),
    Column('alternate name', String(400)),
    Column('isPreferredName', CHAR(1)),
    Column('isShortName', CHAR(1)),    
    Column('isColloquial', CHAR(1)),
    Column('isHistoric', CHAR(1)),
    Column('used_from', CHAR(20)), # should be more than enough
    Column('used_to', CHAR(20)),
)

metadata.create_all(engine)

## The following takes 46 minutes to run! 
#### data.to_sql('alternateNames', con=engine, if_exists='replace', index=False)

674

### `geonames`
This is the main source of data. This table is created from combination of country-specific tables. Here we only take the tables for countries that are of the main customer's interest (see [Project description](#project-descr)) and upload them one by one.

In [41]:
column_names = [
    'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
    'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
    'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
    'dem', 'timezone', 'modification_date'
]

In [83]:
# Reading the text files
data = pd.read_csv('../datasets/RS.txt', sep='\t', names=column_names, encoding='utf-8')


In [ ]:
geonames = Table('geonames', metadata,
    Column('geonameid', Integer),
    Column('name', String(200)),
    Column('asciiname', String(200)),
    Column('alternatenames', String(10000)),
    Column('latitude', DECIMAL),
    Column('longitude', DECIMAL),
    Column('feature_class', CHAR(1)),
    Column('feature_code', String(10)),
    Column('country_code', CHAR(2)),
    Column('cc2', String(200)),
    Column('admin1_code', String(20)),
    Column('admin2_code', String(80)),
    Column('admin3_code', String(20)),
    Column('admin4_code', String(20)),
    Column('population', BIGINT),
    Column('elevation', Integer),
    Column('dem', Integer),
    Column('timezone', String(40)),
    Column('modification_date', DATE)
)
metadata.create_all(engine)

In [84]:
data.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,672867,Moravica,Moravica,"Maravita,Maraviţa,Moravica,Moravicza,Moravita,...",45.23333,21.25000,H,STM,RS,NaN,00,NaN,NaN,NaN,0,NaN,77,Europe/Belgrade,2014-11-05
1,675496,Iron Gates,Iron Gates,"Dealul Klisura,Derdap,Eisenernes Tor,Eisernes ...",44.67965,22.51537,T,GRGE,RS,RO,00,NaN,NaN,NaN,0,NaN,61,Europe/Belgrade,2019-03-01
2,682504,Kazan,Kazan,"Cazane,Cazane Defile,Kazan,Kazan Pass,Kazanske...",44.66667,22.30000,T,GRGE,RS,NaN,00,NaN,NaN,NaN,0,NaN,158,Europe/Belgrade,2014-11-05
3,682722,Râu Caraş,Rau Caras,"Caras,Caraş,Caraș,Karas,Karas River,Karaş,Kara...",44.81667,21.33333,H,STM,RS,RO,00,NaN,NaN,NaN,0,NaN,65,Europe/Belgrade,2021-02-16
4,684724,Kanal Brzava,Kanal Brzava,Kanal Brzava,45.27549,20.82796,H,CNL,RS,NaN,00,NaN,NaN,NaN,0,NaN,72,Europe/Belgrade,2012-07-04


In [85]:
# Upload to the database
data.to_sql('geonames', con=engine, if_exists='append', index=False)

498

In [8]:
# Определяем таблицу для запроса
# geonames = Table('geonames', metadata, autoload_with=engine)

# Test query
query = "SELECT * FROM geonames WHERE country_code = 'RS' LIMIT 10  "
pd.read_sql_query(query, con=engine)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,672867,Moravica,Moravica,"Maravita,Maraviţa,Moravica,Moravicza,Moravita,...",45.23333,21.25000,H,STM,RS,None,00,None,None,None,0,None,77,Europe/Belgrade,2014-11-05
1,675496,Iron Gates,Iron Gates,"Dealul Klisura,Derdap,Eisenernes Tor,Eisernes ...",44.67965,22.51537,T,GRGE,RS,RO,00,None,None,None,0,None,61,Europe/Belgrade,2019-03-01
2,682504,Kazan,Kazan,"Cazane,Cazane Defile,Kazan,Kazan Pass,Kazanske...",44.66667,22.30000,T,GRGE,RS,None,00,None,None,None,0,None,158,Europe/Belgrade,2014-11-05
3,682722,Râu Caraş,Rau Caras,"Caras,Caraş,Caraș,Karas,Karas River,Karaş,Kara...",44.81667,21.33333,H,STM,RS,RO,00,None,None,None,0,None,65,Europe/Belgrade,2021-02-16
4,684724,Kanal Brzava,Kanal Brzava,Kanal Brzava,45.27549,20.82796,H,CNL,RS,None,00,None,None,None,0,None,72,Europe/Belgrade,2012-07-04
5,685194,Begej,Begej,"Bega,Begeiul,Begej,Begheiul,Raul Bega,Riu Bega...",45.20861,20.31528,H,STM,RS,None,00,None,None,None,0,None,71,Europe/Belgrade,2020-08-25
6,686243,Zlatica,Zlatica,"Aranca,Aranka,Zlatica",45.81213,20.14855,H,STM,RS,None,00,None,None,None,0,None,78,Europe/Belgrade,2012-07-04
7,691517,Tisa,Tisa,"Theiss,Theiß,Tisa,Tisza,Tysa",45.13806,20.27750,H,STM,RS,None,00,None,None,None,0,None,69,Europe/Belgrade,2023-09-10
8,725863,Visočica,Visocica,"Visocica,Visočica",43.29663,22.61132,H,STM,RS,None,00,None,None,None,0,None,495,Europe/Belgrade,2012-09-06
9,725902,Vidlich,Vidlich,"Vidlic,Vidlich,Vidlič,Видлич",43.14215,22.80233,T,MTS,RS,None,00,None,None,None,0,None,1329,Europe/Belgrade,2017-03-04


### `cities15000`

In [111]:
column_names = [
    'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
    'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
    'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
    'dem', 'timezone', 'modification_date'
]

data = pd.read_csv('../datasets/cities15000.txt', sep='\t', names=column_names, encoding='utf-8')
data.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2020-03-03
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,62747,NaN,2,Asia/Dubai,2019-10-24
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,351943,NaN,2,Asia/Dubai,2019-09-09
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,NaN,01,103,NaN,NaN,63482,NaN,118,Asia/Dubai,2019-10-24


In [112]:
data.to_sql('cities15000', con=engine, if_exists='append', index=False)

127

In [9]:
metadata = MetaData()

cities = Table('cities15000', metadata,
    Column('geonameid', Integer),
    Column('name', String(200)),
    Column('asciiname', String(200)),
    Column('alternatenames', String(10000)),
    Column('latitude', DECIMAL),
    Column('longitude', DECIMAL),
    Column('feature_class', CHAR(1)),
    Column('feature_code', String(10)),
    Column('country_code', CHAR(2)),
    Column('cc2', String(200)),
    Column('admin1_code', String(20)),
    Column('admin2_code', String(80)),
    Column('admin3_code', String(20)),
    Column('admin4_code', String(20)),
    Column('population', BIGINT),
    Column('elevation', Integer),
    Column('dem', Integer),
    Column('timezone', String(40)),
    Column('modification_date', DATE)
)
# metadata.create_all(engine)

query = select(func.count()).select_from(cities)

# Выполняем запрос и выводим результат
count = pd.read_sql_query(query, con=engine).values[0,0]
print("Number of entries in 'cities15000':", count)

Number of entries in 'cities15000': 27127


In [4]:
# Let's take a look again at what tables are in the database

from sqlalchemy import inspect

inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    #print("schema: %s" % schema)
    print(inspector.get_table_names(schema=schema))

['sql_features', 'sql_implementation_info', 'sql_parts', 'sql_sizing']
['alternateNames', 'geonames', 'countryInfo', 'countryInfo2', 'cities15000']


### `admin1CodesASCII`

In [27]:
column_names = [
    'code', 'name', 'asciiname', 'geonameid'
]

data = pd.read_csv('../datasets/admin1CodesASCII.txt', sep='\t', names=column_names, encoding='utf-8')
data.head()

,code,name,asciiname,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131
3,AD.03,Encamp,Encamp,3040684
4,AD.02,Canillo,Canillo,3041203


In [31]:
metadata = MetaData()

cities = Table('admin1CodesASCII', metadata,
    Column('code', CHAR(5)),
    Column('name', String(200)),
    Column('asciiname', String(100)),
    Column('geonameid', Integer)
)
metadata.create_all(engine)

data.to_sql('admin1CodesASCII', con=engine, if_exists='replace', index=False)

881

# 2. Research

## 2.1 Creating the working dataframe

The dataframe contains the bare minimum info: indiciies (`geonameIDs`) and all possible names.

-> *Upscaling: query* `geonames` *instead of* `cities15000`
-> Unrestricting country selection 

~~The following query asks for the geonameID, country code, names and population in the countries of interest 
and merges this with the data from the table countryInfo to get the country names:~~

In [62]:
country_selection = ('RU', 'KZ', 'AM', 'RS', 'ME', 'KG', 'GE')

# Delete the WHERE clause if you want to select cities from around the globe
query = f'''
SELECT geonameid, name, alternatenames, country_code
FROM cities15000  
WHERE country_code IN {country_selection}
'''
# LEFT JOIN (SELECT "ISO", "Country" FROM "countryInfo") AS ci
# ON cities15000.country_code = ci."ISO"

df = pd.read_sql_query(query, con=engine, index_col = 'geonameid')
df.head()

,name,alternatenames,country_code
geonameid,,,
174875,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",AM
174895,Goris,"Geryusy,Goris,Горис,Գորիս",AM
174972,Hats’avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",AM
174979,Artashat,"Artachat,Artasat,Artasatas,Artasato,Artaschat,...",AM
174991,Ararat,"Ararat,Araratas,Ararato,Davalinskiy Tsemzavod,...",AM


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27127 entries, 3040051 to 1106542
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            27127 non-null  object
 1   alternatenames  24799 non-null  object
dtypes: object(2)
memory usage: 635.8+ KB


## 2.2 Implementing the `fuzzy` search

We will calculate the fuzzy similarity score (the Levenstein distance) of the query string to each and every alternative name. The result is `k` vectors with the highest overall match. 

A metrics tolerant to misspellings is Levinstein distance. The fastest implementation of the computation is implemented in the family of `fuzzy` search. There are various realizaations for Python; here we use [`thefuzz`](https://github.com/seatgeek/thefuzz).
In order to account for different alphabets (the customer is focused on Eastern European and Asian countries that use cyrillic and other non-latin systems) we must make sure that both our reference names and the query are written in with latin symbols. This is easy to do with [`transliterate`](https://pypi.org/project/transliterate/)   

In [13]:
# %pip install thefuzz transliterate

In [63]:
from thefuzz import process
from transliterate import slugify
from transliterate import detect_language
import numpy as np

Now we need to prepare the data

In [120]:
# Splitting the column AlternativeNames into single names: 
altnames = [l.split(',') if l else [None] for l in df.alternatenames.values] 

names=df.name.values

for i in range(len(altnames)):
    altnames[i].append(names[i])
#Creating the dictionary of the structure geonameID: names for all cities including official and alternative names 

d = {zip(df.index, altnames)} # does same as d = {ind: n for ind, n in zip(df.index, altnames)}

Writing the function

In [171]:
def search(query: str, k=10, weight_mode='exp', asdict=True):
    """The rapid fuzzy search
    
    Parameters: 
    - query (str): the query string
    - k (int, optional): Desired number of matches
    - weight_mode={None, 'sq', 'exp'} (str, optional):
        * None: do not weight closer matches
        * 'sq': apply parabolic weighting to the similarity scores
        * 'exp': apply exponential weighting to the similarity scores
    - asdict (bool, optional): whether to convert the result into a python dict. If False, returns Pandas DataFrame """
    
    if detect_language(query) is not None:
        query = slugify(query)
        scores = {} # container for match scores for each city

    for ind, name_list in d.items(): ## for each city calculate similarity scores with every alternative name
        _ = np.array(process.extract(query, name_list)) 
        
        if weight_mode == 'exp':
            # Calculate the scores and weighting them exponentially
            scores[ind] = np.exp(_[:, 1].astype(int)).sum() / len(_) # sum up the exponents of the scores and normalize by the number of possible names
            scores[ind] = np.log(scores[ind]) # return to the readable values
        elif weight_mode == 'sq':
            # Calculate the scores and weighting them parabolically
            scores[ind] = np.square(_[:, 1].astype(int)).sum() / len(_)
            scores[ind] = np.sqrt(scores[ind])
        else: # @TODO: check for the actually None value? 
            scores[ind] = _[:, 1].astype(int).sum() / len(_) 

    # sorted by the matching score (.2 ms faster with the native Python function than with Pandas df.sort_values) see tests below
    scores_df = pd.DataFrame.from_records(
        sorted(scores.items(), key=lambda item: item[1], reverse=True), columns=['geonameid', 'score']) 
    scores_df.loc[:, 'score'] = scores_df.loc[:, 'score'].round(3) # so this looks nicer
    indexes = tuple(scores_df.loc[:k-1, 'geonameid']) # select the DataFrame indicies of the top k
    # print(indexes)

    query = f'''
        SELECT DISTINCT
            cities.geonameid,
            cities.name,
            regions.name as region,
            ci."Country" as country

        FROM
            cities15000 AS cities
        LEFT JOIN
            (SELECT "ISO", "Country" FROM "countryInfo") AS ci
        ON
            cities.country_code = ci."ISO"
        LEFT JOIN
            "admin1CodesASCII" AS regions
        ON
            COALESCE(cities.country_code, '') || '.' || COALESCE(cities.admin1_code, '') = regions.code
        WHERE
            cities.geonameid IN {indexes};
    '''

    qres = pd.read_sql_query(query, con=engine)
    result = pd.merge(qres, scores_df, on='geonameid', how='left').sort_values('score', ascending=False).set_index('geonameid')
    if asdict:
        return result.T.to_dict()
    else:
        return result

Testing: 

Misspelled name

In [174]:
%%time
k=3 ## number of suggestions
query = 'Ржевск'

search(query, k, weight_mode=None, asdict=False)

CPU times: total: 188 ms
Wall time: 543 ms


,name,region,country,score
geonameid,,,,
554840,Izhevsk,Udmurtiya Republic,Russia,80.0
499717,Rzhev,Tver Oblast,Russia,77.6
518659,Novokuybyshevsk,Samara Oblast,Russia,75.0


Historical name

In [175]:
%%time
k=10 ## number of suggestions
#query = 'Сталинград'
query = 'Атомград'

result = search(query, k, weight_mode=None, asdict=False)
result

CPU times: total: 188 ms
Wall time: 570 ms


,name,region,country,score
geonameid,,,,
566199,Dimitrovgrad,Ulyanovsk,Russia,68.0
485698,Svetlograd,Stavropol Kray,Russia,67.0
792680,Belgrade,Central Serbia,Serbia,62.0
1489425,Tomsk,Tomsk Oblast,Russia,60.0
610529,Atyrau,Atyraū,Kazakhstan,59.8
463828,Zelenogradsk,Kaliningrad Oblast,Russia,59.4
472757,Volgograd,Volgograd Oblast,Russia,59.4
554234,Kaliningrad,Kaliningrad Oblast,Russia,58.8
2027456,Artëm,Primorye,Russia,58.8


In [176]:
%%time
k=10 
query = 'Атомград'

result = search(query, k, weight_mode='exp', asdict=False)
result

CPU times: total: 172 ms
Wall time: 535 ms


,name,region,country,score
geonameid,,,,
1538635,Zheleznogorsk,Krasnoyarsk Krai,Russia,98.391
3193044,Podgorica,Podgorica,Montenegro,73.391
1526038,Atbasar,Aqmola,Kazakhstan,70.391
1540356,Raduzhny,Khanty-Mansia,Russia,70.391
2027968,Aldan,Sakha,Russia,70.391
498817,Saint Petersburg,St.-Petersburg,Russia,69.409
610529,Atyrau,Atyraū,Kazakhstan,69.391
566199,Dimitrovgrad,Ulyanovsk,Russia,68.000
557469,Igra,Udmurtiya Republic,Russia,67.595


As such, the search works a bit better with the rare and historicsl names when the weighting is applied.

Exponential weighting works far better, and the square is just not enough. 
The result is awesome! We are returning the most likely matches, and surfacing out the precise ones, if any. 

# Some tests

In [178]:
test_df = pd.read_csv('../datasets/geo_test.csv', sep=';')
test_df.head()

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan
3,Москва,Moscow,Moscow,Russia
4,Алматы,Almaty,Almaty,Kazakhstan


In [179]:
queries = test_df['query'].values

In [181]:
%%time

for q in queries:  
    search(q, 10, weight_mode='exp', asdict=True)

CPU times: total: 1min
Wall time: 3min 5s


Processsing of 346 queries took 3 mins 5 s. 

@TODO: measure accuracy!

### Further performance tests

In [150]:
%%timeit

pd.DataFrame.from_records(list(scores.items()), columns=['geonameid', 'score']).sort_values('score', ascending=False)

1.9 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
%%timeit

pd.DataFrame.from_records(
    sorted(scores.items(), key=lambda item: item[1], reverse=True), columns=['geonameid', 'score'])

1.74 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


it is a little bit faster to sort the scores before creating the DataFrame

----------------------------------

# Demo of the assembly

In [11]:
from geonamesearch import * 

In [6]:
q = 'Karaganda'
search(q, k=10, weight_mode='exp', asdict=False)

Connection established: geo_v2 на 77.222.36.33


,name,region,country,score
geonameid,,,,
609655,Karagandy,Karaganda,Kazakhstan,98.391
2028078,Aginskoye,Zabaykalskiy (Transbaikal) Kray,Russia,88.391
790015,Inđija,Vojvodina,Serbia,78.391
1501321,Kurgan,Kurgan Oblast,Russia,73.457
553915,Kaluga,Kaluga Oblast,Russia,73.391
552977,Karabanovo,Vladimir Oblast,Russia,72.391
551964,Kashira,Moscow Oblast,Russia,72.000
523064,Nazran’,Ingushetiya Republic,Russia,70.391
1488253,Zelenogorsk,Krasnoyarsk Krai,Russia,70.391


In [4]:
q = 'Milkyway'
search(q, k=3, weight_mode='exp', asdict=False)

,name,region,country,score
geonameid,,,,
561731,Gay,Orenburg Oblast,Russia,71.084
527740,Melenki,Vladimir Oblast,Russia,70.391
1526193,Arkalyk,Qostanay,Kazakhstan,70.391


In [10]:
q = 'Стфлинград'
search(q, k=3, weight_mode='exp', asdict=False)

,name,region,country,score
geonameid,,,,
472757,Volgograd,Volgograd Oblast,Russia,88.427
1526273,Astana,Astana,Kazakhstan,74.393
498817,Saint Petersburg,St.-Petersburg,Russia,72.409


In [7]:
q = 'Россия'
search(q, k=3, weight_mode='exp', asdict=False)

,name,region,country,score
geonameid,,,,
1527534,Osh,Osh,Kyrgyzstan,89.084
514706,Osa,Perm Krai,Russia,88.391
518255,Novorossiysk,Krasnodar Krai,Russia,76.489


In [8]:
q = 'Ржевск'
search(q, k=3, weight_mode='exp', asdict=False)

,name,region,country,score
geonameid,,,,
554840,Izhevsk,Udmurtiya Republic,Russia,85.085
499717,Rzhev,Tver Oblast,Russia,82.085
1528121,Karakol,Issyk-Kul,Kyrgyzstan,77.084


In [7]:
q = 'Berlin'
search(q, k=3, weight_mode='exp', asdict=False, country_selection=None)

,name,region,country,score
geonameid,,,,
2950159,Berlin,Berlin,Germany,99.084
5164706,North Canton,Ohio,United States,98.391
2820577,Überlingen,Baden-Wurttemberg,Germany,90.000


In [10]:
q = 'Berlin'
search(q, k=3, weight_mode='exp', asdict=False, country_selection=('DE', 'KZ'))

,name,region,country,score
geonameid,,,,
2950159,Berlin,Berlin,Germany,99.084
2820577,Überlingen,Baden-Wurttemberg,Germany,90.000
2950096,Bernau bei Berlin,Brandenburg,Germany,89.777


In [27]:
q = 'Berlin'
search(q, k=3, weight_mode='exp', asdict=False, country_selection=['DE'])

Connection established: geo_v2 на 77.222.36.33

    SELECT geonameid, name, alternatenames, country_code
    FROM cities15000  
    WHERE country_code = 'DE'
    
['DE'] 1


,name,region,country,score
geonameid,,,,
2950159,Berlin,Berlin,Germany,99.084
2820577,Überlingen,Baden-Wurttemberg,Germany,90.000
2950096,Bernau bei Berlin,Brandenburg,Germany,89.777
